<a href="https://colab.research.google.com/github/Deconer/bih_upd/blob/master/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22Ennro_hr_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# НЕЙРОСЕТИ И ТРЕЙДИНГ. ОБУЧЕНИЕ И ТЕСТИРОВАНИЕ НЕЙРОСЕТИ

⚠️️ ***Есть новая [статья](https://habr.com/ru/post/562092/) и [новый CoLab](https://colab.research.google.com/drive/1VSKckyGLA-1SAgGFkoaENJCB6lOgcnBs?usp=sharing) !!!***


#1. УСТАНОВКА БИБЛИОТЕК

In [ ]:
!pip install PrettyTable
!pip install matplotlib
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Dropout, Activation
from tensorflow.python.keras import utils
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os.path, zipfile, shutil, requests, urllib.request
from google.colab import files
from IPython.display import clear_output
from prettytable import PrettyTable
from datetime import datetime


def make_model(n, dropout, act):
    model = Sequential()
    model.add(Dense(n, input_dim=n, activation=act))
    model.add(Dropout(dropout))
    model.add(Dense(int(n * 0.5), activation=act))
    model.add(Dropout(dropout))
    model.add(Dense(2, activation="softmax"))
    model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=['accuracy'])
    return model


def get_files():
    for file_object in os.listdir('/content'):
        file_object_path = os.path.join('/content', file_object)
        if os.path.isfile(file_object_path):
            os.unlink(file_object_path)
        else:
            shutil.rmtree(file_object_path)
    print("⚠️️ Идет загрузка данных...")
    f = requests.get("https://torgap.ru/Files.zip", allow_redirects=True)
    open('Files.zip', 'wb').write(f.content)
    if os.path.exists("Files.zip"):
        clear_output(wait=True)
        print("✅️ Данные загружены, обрабатываются...\n")
        with zipfile.ZipFile("/content/Files.zip", "r") as zip_ref:
            zip_ref.extractall()
        os.unlink('/content/Files.zip')
        f_list = os.listdir('/content/')
        for i, data in enumerate(f_list):
            st = data[:-4]
            st = st.split("_")
            symbol = st[1]
            dim = st[2]
            timeframe = st[3]
            break
        clear_output(wait=True)
        return symbol, dim, timeframe
    else:
        print("⚠️️ Проблема с загрузкой, что то пошло не так.")
        raise SystemExit


def make_data(f_name):
    data = np.loadtxt("/content/" + f_name + "_" + symbol + "_" + dim + "_" +
                      timeframe + ".txt", delimiter=",", encoding="utf-16")
    size = data.shape[1] - 4
    qtt = data.shape[0]
    x = data[1:qtt, 0:size]
    y = data[1:qtt, size:(size + 2)]
    cndls = data[1:qtt, size:(size + 4)]
    print("✅️ " + f_name + " данных: " + str(qtt) + ", " + f_name + " предикторов: " + str(size) + "\n")
    if f_name == "test" and qtt < 10000: print(
        "⚠️️ Очень мало данных для тестирования!!! Должно быть не меньше 10'000 !!!")
    if f_name == "train" and qtt < (size * 2000): print(
        "⚠️️ Очень мало данных для обучения!!! Должно быть не меньше " + str(size * 2000) +
        "!!! Минимум по 2000 на каждый предиктор.")
    return x, y, size, qtt, cndls


def test_result(model, test_arr):
    a = np.empty(shape=[qtt_test - 1, 1])
    p = model.predict(test_arr)
    x = PrettyTable()
    best_res = 0
    tn = 0
    x.field_names = ['Ответ сети', 'Выиграл', 'Проиграл', 'Выиграл (%)', 'Сигналов (%)']
    for i, pr in enumerate(p): a[i] = int(round(((pr[0] + tn) - (pr[1] - tn)) * 100, 0))
    results = np.hstack([a, cndls_test])
    for lim in range(0, 99, 2):
        win = lose = 0
        for i, d in enumerate(results):
            if abs(d[0]) >= lim:
                if (d[0] > 0 and d[1] == 1) or (d[0] < 0 and d[1] == 0):
                    win += 1
                else:
                    lose += 1
        if win + lose != 0:
            res = int((win / (win + lose)) * 100)
        else:
            res = 0
        trades = round(((win + lose) / results.shape[0]) * 100, 1)
        if trades < 1: break
        # if best_res < res:
        best_res = res
        best_lim = lim
        best_trades = trades
        x.add_row([lim, win, lose, res, trades])
    if best_res <= n_lim:
        clear_output(wait=True)
        print("⚠️️ Результат <", n_lim, ". Очень низкое качество обучения." +
              "\nПовторите обучение просто перезапустив этот блок.")
        print("Заново загружать данные не нужно.")
    else:
        print(x)
        print("\n✅️ Последнее значение -", str(best_res) + "% правильных ответов.\n" +
              'Этот результат будет получен если брать ответы с "уверенность" больше', best_lim, "(Ответ сети).",
              "\nСигналов будет -", str(best_trades) +
              "% от общего количества свечей которые подавались на тестирование.")
    return best_lim, best_res, results


def plot_res(step=0):
    if step == 0: step = qtt_test
    if best_res >= n_lim:
        t = 1
        for j in range(step, qtt_test + 1, step):
            u = d = np.empty(shape=[0, 2])
            for i, r in enumerate(results[t:j, ::]):
                if r[0] >= best_lim:
                    u = np.append(u, [[r[3], r[4]]], axis=0)
                if r[0] <= -best_lim:
                    d = np.append(d, [[r[3], r[4]]], axis=0)
            plt.figure(num=None, figsize=(30, 12), dpi=120, facecolor='w', edgecolor='k')
            plt.plot(cndls_test[t:j, 2:3], cndls_test[t:j, 3:4], color='lightgray')
            plt.plot(u[::, 0:1], u[::, 1:2], "go")
            plt.plot(d[::, 0:1], d[::, 1:2], "ro")
            t = j
    else:
        print("⚠️️ Качество обучения слижком низкое, нечего рисовать.")


def fit_model():
    model = make_model(size_train, dropout, act)
    print("Идет обучение...")
    history = model.fit(x_train, y_train, epochs=epoch, validation_split=0.2, batch_size=128, verbose=0)
    clear_output(wait=True)
    print("✅️ Обучение модели завершено\n")
    return model


def get_from_ennro(symbol, tfm, dim, lim):
    if symbol != "BTCUSD" and symbol != "EURUSD":
        print("\n⚠️️ Проверка реальным рынком возможна только для BTCUSD или EURUSD.\n")
        raise SystemExit
    tn = 0
    r = requests.post("https://api.torgap.ru/live.php", data={'symbol': symbol, 'tf': tfm, 'dim': str(dim)}).text
    d = r.split("|")
    size = np.fromstring(d[0], sep=",").shape[0]
    data = np.empty(shape=[1, size])
    # print(data)
    for dt in d:
        m = np.array([np.fromstring(dt, sep=",")])
        data = np.append(data, m, axis=0)
    p = model.predict(data[1:, 0:(size - 2)])
    a = np.empty(shape=[p.shape[0], 1])
    for i, pr in enumerate(p):
        a[i] = int(round(((pr[0] + tn) - (pr[1] - tn)) * 100, 0))
    results = np.hstack([a, data[1:, (size - 2):size]])
    u = d = np.empty(shape=[0, 2])
    for i, r in enumerate(results):
        if r[0] >= lim:
            u = np.append(u, [[r[1], r[2]]], axis=0)
        if r[0] <= -lim:
            d = np.append(d, [[r[1], r[2]]], axis=0)
    print("Отправлен запрос:", symbol, timeframe, "dim:", dim, "\n✅️ Получено", data.shape[0],
          "свечей, последняя свеча", timeframe, "открылась в",
          datetime.utcfromtimestamp(int(data[1][size - 2])).strftime('%H:%M %d-%m-%Y'), "(UTC)\n")
    if u.shape[0] == 0 and d.shape[0] == 0:
        print('⚠️️ Сигналов нет при best_lim =', lim, ". Возможно сейчас маленькая волатильность.\n" +
              "Попробуйте заметить best_lim на значение меньшее чем", lim, "\n")
    plt.figure(num=None, figsize=(30, 12), dpi=120, facecolor='w', edgecolor='k')
    plt.plot(data[1:, (size - 2):(size - 1)], data[1:, (size - 1):(size - 0)], color='lightgray')
    plt.plot(u[::, 0:1], u[::, 1:2], "go")
    plt.plot(d[::, 0:1], d[::, 1:2], "ro")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#2. ЗАГРУЗКА И ПОДГОТОВКА ДАННЫХ ДЛЯ ОБУЧЕНИЯ

In [ ]:
symbol, dim, timeframe = get_files()
x_train, y_train, size_train, qtt_train, cndls_train = make_data("train")
x_test, y_test, size_test, qtt_test, cndls_test = make_data("test")

⚠️️ Идет загрузка данных...


ConnectionError: ignored

#3. ОБУЧЕНИЕ И ТЕСТИРОВАНИЕ МОДЕЛИ
💡 Этот шаг и следующие можно повторять без предыдущих шагов.

In [ ]:
epoch = 10 # количество эпох
act = "tanh" # функция активации ("relu" или "tanh")
dropout = 0.2 # от 0 до 0.5

j = 5 # количество проходов
n_lim = 60 # минимальный допустимый процент правильных ответов при тестировании
for i in range(0, j):
    print("Проход", i+1, "из", j)
    model = fit_model() # обучение
    best_lim, best_res, results = test_result(model, x_test) # тестирование
    if best_res >= n_lim: break    
    clear_output(wait=True)

# 4. РЕЗУЛЬТАТЫ НА ТОРГОВОМ ГРАФИКЕ (все на одном)

In [ ]:
# КРАСНАЯ точка - прогноз вниз, ЗЕЛЕНАЯ - прогноз вверх.
plot_res()

# 5. РЕЗУЛЬТАТЫ НА ТОРГОВОМ ГРАФИКЕ (детально)

In [ ]:
# 1500 свечей на один график
plot_res(1500)

# 6. ПРОВЕРКА МОДЕЛИ НА РЕАЛЬНОМ РЫНКЕ
💡 Модель будет протестирована на данных о послених свечах на реальном рынке (до 1000 свечей)

⚠️ Проверка реальным рынком может не работать в связи с тем, что не всегда запущен сервер обслуживающий этот сервис. Если не работает попробуйте зайти позже.

In [ ]:
lim = best_lim # тут менять best_lim если сигналов нет
get_from_ennro(symbol, timeframe, dim, lim)